In [1]:
import numpy as np
import tensorflow as tf
import json
import sys, os
import itertools
from tqdm import tqdm

path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(path)
np.set_printoptions(formatter={'float': '{:4.4f}'.format})

from utils.utils import forward_kinematics, forward_kinematics_TF
from utils.dh_params import dh_params

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def get_home_position():
    """Calculate the home position (all thetas are 0)"""
    home_thetas = tf.zeros((1, 6), dtype=tf.float32)
    home_positions_orientations, _ = forward_kinematics_TF(home_thetas, dh_params)
    return home_positions_orientations[0, -1].numpy()  # Return end effector position

def generate_dataset_custom_ranges(joint_angle_ranges, position_range=None, batch_size=1000):
    """
    Generate a dataset using custom ranges and resolutions for each joint angle.
    Only include data points where the end effector position (after subtracting home position) 
    is within the specified range. Uses TensorFlow for faster computation.
    
    Args:
        joint_angle_ranges (list of dict): List of dictionaries specifying the range and resolution for each joint
                                           e.g., [{'min': -75, 'max': 75, 'resolution': 1}, ...]
        position_range (dict): Dictionary specifying the range for each axis
                               e.g., {'x': (-100, 100), 'y': (-100, 100), 'z': (0, 200)}
        batch_size (int): Number of combinations to process at once
    
    Returns:
        dict: Dataset containing inputs and outputs
    """
    # Create the range of angles for each joint
    angle_ranges = [np.arange(joint['min'], joint['max'] + joint['resolution'], joint['resolution']) 
                    for joint in joint_angle_ranges]
    
    # Initialize lists to store valid data points
    valid_inputs = []
    valid_outputs = []
    
    # Set default position range if not provided
    if position_range is None:
        position_range = {'x': (-np.inf, np.inf), 'y': (-np.inf, np.inf), 'z': (-np.inf, np.inf)}
    
    # Calculate home position
    home_position = get_home_position()
    
    # Calculate total number of combinations for the progress bar
    total_combinations = np.prod([len(range) for range in angle_ranges])
    
    # Generate combinations of joint angles and calculate forward kinematics
    combinations = itertools.product(*angle_ranges)
    
    for i in tqdm(range(0, total_combinations, batch_size), desc="Generating dataset"):
        # Get a batch of combinations
        batch = list(itertools.islice(combinations, batch_size))
        if not batch:
            break
        
        # Convert batch to TensorFlow tensor
        thetas_batch = tf.constant(batch, dtype=tf.float32)
        
        # Calculate forward kinematics for the batch
        positions_orientations, _ = forward_kinematics_TF(thetas_batch, dh_params)
        end_effectors = positions_orientations[:, -1, :]
        
        # Subtract home position
        end_effectors_relative = end_effectors - home_position
        
        # Check if end effector positions are within the specified range
        x, y, z = end_effectors_relative[:, :3].numpy().T
        mask = (
            (position_range['x'][0] <= x) & (x <= position_range['x'][1]) &
            (position_range['y'][0] <= y) & (y <= position_range['y'][1]) &
            (position_range['z'][0] <= z) & (z <= position_range['z'][1])
        )
        
        valid_inputs.extend(np.array(batch)[mask])
        valid_outputs.extend(end_effectors_relative.numpy()[mask])
    
    return {
        'inputs': np.array(valid_inputs),
        'outputs': np.array(valid_outputs)
    }

def analyze_joint_ranges(dataset):
    """
    Analyze the dataset to find the minimum and maximum angles for each joint.
    
    Args:
        dataset (dict): Dataset containing 'inputs' (joint angles)
    
    Returns:
        dict: Minimum and maximum angles for each joint
    """
    inputs = dataset['inputs']
    num_joints = inputs.shape[1]
    
    joint_ranges = {}
    for i in range(num_joints):
        joint_ranges[f'joint_{i+1}'] = {
            'min': np.min(inputs[:, i]),
            'max': np.max(inputs[:, i])
        }
    
    return joint_ranges

In [ ]:
# Define custom ranges for each joint
joint_angle_ranges = [
    {'min': -60, 'max': 60, 'resolution': 5},  # Joint 1
    {'min': -60, 'max': 60, 'resolution': 5},  # Joint 2
    {'min': -60, 'max': 60, 'resolution': 5},  # Joint 3
    {'min': -60, 'max': 60, 'resolution': 5},  # Joint 4
    {'min': -60, 'max': 60, 'resolution': 10},  # Joint 5
    {'min': -60, 'max': 60, 'resolution': 10}  # Joint 6
]

RNG = 40
# Define the position range for the end effector
position_range = {
    'x': (-RNG, RNG),  # X range from -100 to 100
    'y': (-RNG, RNG),  # Y range from -100 to 100
    'z': (-RNG, RNG)      # Z range from 0 to 200
}

# Generate dataset
print("Generating dataset...")
dataset = generate_dataset_custom_ranges(
    joint_angle_ranges,
    position_range,
    batch_size=2**18
)

print(f"\nDataset size: {len(dataset['inputs'])} samples")

Generating dataset...


Generating dataset:  13%|█▎        | 8/63 [00:08<00:56,  1.02s/it]

In [ ]:
# Print some statistics
print("\nDataset statistics:")
print(f"Input shape: {dataset['inputs'].shape}")
print(f"Output shape: {dataset['outputs'].shape}")

In [ ]:
# Analyze joint ranges
actual_joint_ranges = analyze_joint_ranges(dataset)

print("\nActual joint angle ranges that produce end effector positions within the specified limits:")
for joint, range_info in actual_joint_ranges.items():
    print(f"{joint}: {range_info['min']:.2f} to {range_info['max']:.2f} degrees")

In [ ]:
# Print the first few samples
print("\nFirst few samples:")
for i in range(3):
    print(f"\nSample {i + 1}:")
    print(f"Input (joint angles): {dataset['inputs'][i]}")
    print(f"Output (end effector position relative to home): {dataset['outputs'][i][:3]}")
    print(f"Output (end effector orientation): {dataset['outputs'][i][3:]}")